In [1]:
import requests as req
import pandas as pd
import sqlite3

In [2]:
ENDPOINT = 'https://api.hh.ru/vacancies'
DB_NAME = 'api.db'
TABLE_NAME = 'vacancies'
LIMIT_OF_VACANCY_DETAIL = 100

In [3]:

api_params = {'text':'python middle',
              'per_page':25,
              'page':0}
vacancyDetailInfo =[]

In [4]:
def parsJsonAppendData (data):
    for result in data['items']:
        vacancyRow = []
        listOfRequirements = []
        vacancyName = result['name']
        companyName = result['employer']['name']
        vacancyid = result['id']
        vacancyDetail = req.get(ENDPOINT+'/'+vacancyid).json()
        vacancyDescription = vacancyDetail['description']
        for skill in vacancyDetail['key_skills']:
            listOfRequirements.append(skill['name'])
        stringOfRequirements = ', '.join(listOfRequirements)
        vacancyRow.append(companyName)
        vacancyRow.append(vacancyName)
        vacancyRow.append(vacancyDescription)
        vacancyRow.append(stringOfRequirements)
        vacancyDetailInfo.append(vacancyRow)

In [5]:
while (True):
    results = req.get(ENDPOINT, params = api_params).json()
    if (len(results['items']) == 0 or len(vacancyDetailInfo)>=LIMIT_OF_VACANCY_DETAIL):
        break
    parsJsonAppendData(results)
    print(f'Список детальной информации о вакансиях содержит {len(vacancyDetailInfo)} записей')
    api_params['page'] += 1
    

Список детальной информации о вакансиях содержит 25 записей
Список детальной информации о вакансиях содержит 50 записей
Список детальной информации о вакансиях содержит 75 записей
Список детальной информации о вакансиях содержит 100 записей


In [6]:
table_columns = ['company_name', 'position', 'job_description', 'key_skills']
df = pd.DataFrame(data=vacancyDetailInfo,columns=table_columns,index=None)

In [8]:
connection = sqlite3.connect(DB_NAME)
df.to_sql(TABLE_NAME,connection,if_exists='replace',index=False)
connection.close()